## Around the World in 80 days

In [1]:
!python -m spacy download en_core_web_sm
import pandas as pd
pd.set_option('display.max_colwidth', 800)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 11.1 MB 1.7 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=81dd1d52469a4df06270cc21c72519d9f43e05ab1a0658fdad1610735df32181
  Stored in directory: /tmp/pip-ephem-wheel-cache-0_ictqt3/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
OPEN_DATA_URL = '../data/aroundtheworld/pg103.txt'
f=open(OPEN_DATA_URL,'r',encoding='utf-8')
text=f.read()
f.close()

import re

HANDLE = '@\w+'
LINK = 'https?://t\.co/\w+'
SPECIAL_CHARS = '&lt;|&lt;|&amp;|#'
PARA='\n+'
def clean(text):
    #text = re.sub(HANDLE, ' ', text)
    text = re.sub(LINK, ' ', text)
    text = re.sub(SPECIAL_CHARS, ' ', text)
    text = re.sub(PARA, '\n', text)
    return text

text = clean(text)
text

'\ufeffThe Project Gutenberg EBook of Around the World in 80 Days, by Jules Verne\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.net\nTitle: Around the World in 80 Days\nAuthor: Jules Verne\nRelease Date: May 15, 2008 [EBook  103]\nLast updated: February 18, 2012\nLast updated: May 5, 2012\nLanguage: English\n*** START OF THIS PROJECT GUTENBERG EBOOK AROUND THE WORLD IN 80 DAYS ***\nAROUND THE WORLD IN EIGHTY DAYS\nCONTENTS\nCHAPTER\n      I  IN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER, THE\n         ONE AS MASTER, THE OTHER AS MAN\n     II  IN WHICH PASSEPARTOUT IS CONVINCED THAT HE HAS AT LAST FOUND\n         HIS IDEAL\n    III  IN WHICH A CONVERSATION TAKES PLACE WHICH SEEMS LIKELY TO COST\n         PHILEAS FOGG DEAR\n     IV  IN WHICH PHILEAS FOGG ASTOUNDS PASSEPARTOUT, 

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc=nlp(text)
pos_list=['NOUN']

preproc_text=[]
preproc_sent=[]

for token in doc:
    if token.text!='\n':
        if not(token.is_stop) and not(token.is_punct) and token.pos_ in pos_list:
            preproc_sent.append(token.lemma_)
    else:
        preproc_text.append(preproc_sent)
        preproc_sent=[]

preproc_text.append(preproc_sent) #last sentence

print(preproc_text)

[['day'], ['use', 'cost'], ['restriction'], ['term'], ['www.gutenberg.net'], ['title', 'day'], ['author'], ['release', 'date'], [], [], ['language'], ['start', 'project', 'world', 'day'], ['world', 'day'], ['contents'], ['chapter', 'convinced', 'ideal', 'conversation', 'place', 'servant', 'v', 'species', 'IMPATIENCE', 'talks', 'ix', 'design', 'x', 'GLAD', 'loss', 'means', 'CONVEYANCE', 'price', 'xii', 'PROOF', 'BRAVE', 'xiv', 'LENGTH', 'VALLEY', 'gange', 'banknote', 'thousand', 'voyage', 'interest', 'xx', 'face', 'master', 'runs', 'risk', 'losing', 'reward', 'money', 'xxiv', 'glimpse', 'party', 'travel', 'speed', 'miles', 'course', 'xxviii', 'passepartout', 'incident', 'further', 'interest', 'struggle', 'shows', 'occasion', 'reaches', 'order', 'premium', 'world'], ['chapter'], [], [], [], ['house'], ['member'], ['attention', 'personage'], ['man', 'world', 'People'], ['head'], ['year'], [], [], ['change'], ['counting', 'room', 'ship', 'dock'], ['owner', 'employment'], [], ['inn', 'inn',

In [4]:
import tomotopy as tp
mdl = tp.HDPModel(min_cf=5,seed=0)

for line in preproc_text:
    mdl.add_doc(line)

for i in range(0, 100, 10):
    mdl.train(i)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=7))

Iteration: 0	Log-likelihood: -6.954251056649627
Iteration: 10	Log-likelihood: -6.578738653168456
Iteration: 20	Log-likelihood: -6.5483394866180324
Iteration: 30	Log-likelihood: -6.769227106027983
Iteration: 40	Log-likelihood: -7.100002631415033
Iteration: 50	Log-likelihood: -7.300310460533384
Iteration: 60	Log-likelihood: -7.307419362533066
Iteration: 70	Log-likelihood: -7.291183359824084
Iteration: 80	Log-likelihood: -7.263182028165088
Iteration: 90	Log-likelihood: -7.332802224573655
Top 10 words of topic #0
[('steamer', 0.06810998916625977), ('day', 0.055888254195451736), ('boat', 0.034936707466840744), ('board', 0.03144478425383568), ('pilot', 0.03144478425383568), ('sail', 0.029698822647333145), ('course', 0.029698822647333145)]
Top 10 words of topic #1
[('train', 0.08387869596481323), ('time', 0.06291276216506958), ('station', 0.05392736569046974), ('hour', 0.043444402515888214), ('minute', 0.038951702415943146), ('car', 0.035956572741270065), ('master', 0.02697117254137993)]
Top 

In [5]:
bag_of_words=[word for sent in preproc_text for word in sent]
doc_inst = mdl.make_doc(bag_of_words)
np.argsort(np.array(mdl.infer(doc_inst)[0]))[::-1]

array([329,  81, 116, 124,  14,  26, 105, 177, 244,  28,  77, 304, 240,
       104,  38,  27,   2, 193, 259,  73,   1,   0,  56, 298, 310, 322,
       200, 305, 164,  97, 197,  80,  72, 204,  68, 165, 249, 145, 108,
         3,  15,  71,  91, 223,  83,  75,  59,  67,  63,  99, 297, 243,
       247, 215,  79, 207, 107, 309, 203, 111, 115, 199, 119, 195, 123,
       127, 179, 147, 167, 251, 159, 321,  13, 299, 323,  57, 311,  43,
       314, 132, 131, 130, 129, 128,  16,  17, 134,  18,  19, 327, 315,
        20, 326,  21,  22,  23,  24,  25, 133, 136, 135, 149, 155, 154,
        10, 306,  11, 152, 151, 150, 148, 137,  12, 143, 142, 141, 140,
       325, 139, 138, 324, 101, 103,  52, 160,  35,  36,  37,  55,  54,
        39,   6,  53,  40, 102,  51,  41,  50,  42,  49,  48,  47,  46,
        45,  34, 320,  33, 319,  44, 100,   7,   4,  95,  29,  94,  93,
        92,  30,  87,  86,  85,  31,  84,  32, 318,   5,   9,   8, 307,
       161, 260, 253, 254, 255, 256, 257, 258, 261, 218, 262, 26

In [6]:
print(mdl.get_topic_words(33, top_n=7))

[('player', 0.11542856693267822), ('cry', 0.11542856693267822), ('second', 0.11542856693267822), ('terror', 0.0011428571306169033), ('rogue', 0.0011428571306169033), ('glimpse', 0.0011428571306169033), ('means', 0.0011428571306169033)]


In [7]:
print(mdl.get_topic_words(21, top_n=7))

[('deck', 0.11153203248977661), ('detective', 0.0781058520078659), ('seat', 0.066963791847229), ('cabin', 0.055821727961301804), ('lady', 0.055821727961301804), ('state', 0.0446796678006649), ('heart', 0.0446796678006649)]


In [8]:
print(mdl.get_topic_words(70, top_n=7))

[('country', 0.09241379052400589), ('Brindisi', 0.09241379052400589), ('bet', 0.09241379052400589), ('discussion', 0.04643678292632103), ('tambourine', 0.04643678292632103), ('traveller', 0.04643678292632103), ('seat', 0.04643678292632103)]


In [9]:
print(mdl.get_topic_words(4, top_n=7))

[('chapter', 0.6029223203659058), ('torch', 0.03671232983469963), ('manner', 0.03671232983469963), ('volunteer', 0.01844748854637146), ('satisfaction', 0.01844748854637146), ('violence', 0.01844748854637146), ('means', 0.01844748854637146)]
